In [1]:
import numpy as np
import copy
from colorama import Fore, Back, Style

In [2]:
class colors:
    reset='\033[0m'
    bold='\033[01m'
    disable='\033[02m'
    underline='\033[04m'
    reverse='\033[07m'
    strikethrough='\033[09m'
    invisible='\033[08m'
    class fg:
        black='\033[30m'
        red='\033[31m'
        green='\033[32m'
        orange='\033[33m'
        blue='\033[34m'
        purple='\033[35m'
        cyan='\033[36m'
        lightgrey='\033[37m'
        darkgrey='\033[90m'
        lightred='\033[91m'
        lightgreen='\033[92m'
        yellow='\033[93m'
        lightblue='\033[94m'
        pink='\033[95m'
        lightcyan='\033[96m'
    class bg:
        black='\033[40m'
        red='\033[41m'
        green='\033[42m'
        orange='\033[43m'
        blue='\033[44m'
        purple='\033[45m'
        cyan='\033[46m'
        lightgrey='\033[47m'

In [3]:
def print_board(board):
    print(colors.bg.black + colors.fg.lightblue+ "   A   B   C   D   E   F   G   H   ")
    count = 1
    for row in board:
        print(colors.fg.lightblue + str(count), end="  ")
        count+=1
        for cell in row:
            if cell == 0:
                print(colors.fg.green + "█", end="   ")
            if cell == 1:
                print(colors.fg.lightgrey + "█", end="   ")
            if cell == -1:
                print(colors.fg.black + "█", end="   ")
            if cell == 3:
                print(colors.fg.green + "░", end="   ")
        print("")

In [4]:
class State:
    def __init__(self, *input):
        self.origin = []
        self.destin = None
        self.heuristic = 0
        self.pieces_taken = []
        self.board = input[0]
        self.utility = 0
        if len(input)>1:
            self.origin = input[0]
            self.destin = input[1]
            self.heuristic = input[2]
            self.pieces_taken = input[3]
            self.board = input[4]
        
    def list_actions(self,player):
        token = 1
        if player == "min" : token = -1
        enemy_indexes = np.where(self.board == token)
        enemy_coordinates= list(zip(enemy_indexes[0], enemy_indexes[1]))
        return enemy_coordinates

    def print_board_changes(self):
        if type(self.origin) is list:
            player_piece = self.board[self.origin[0][0]][self.origin[0][1]]
        else:
            player_piece = self.board[self.origin[0]][self.origin[1]]
        for piece in self.pieces_taken:
            self.board[piece[0]][piece[1]] = player_piece
        self.board[self.destin[0]][self.destin[1]] = player_piece

    def can_move_up(self, piece, board, player_piece):# 1 -> y  0 -> x
        if piece[1] == 0: return False
        if board[piece[0]][piece[1]-1] == player_piece: return False
        if board[piece[0]][piece[1]-1] == player_piece*-1: return "Enemy"
        return "Empty" #enemy found?

    def can_move_down(self, piece, board, player_piece):
        if piece[1] == 7: return False
        if board[piece[0]][piece[1]+1] == player_piece: return False
        if board[piece[0]][piece[1]+1] == player_piece*-1: return "Enemy"
        return "Empty"
    
    def can_move_left(self, piece, board, player_piece):
        if piece[0] == 0: return False
        if board[piece[0]-1][piece[1]] == player_piece: return False
        if board[piece[0]-1][piece[1]] == player_piece*-1: return "Enemy"
        return "Empty"

    def can_move_right(self, piece, board, player_piece):
        if piece[0] ==7 : return False
        if board[piece[0]+1][piece[1]] == player_piece: return False
        if board[piece[0]+1][piece[1]] == player_piece*-1: 
            return "Enemy"
        return "Empty"
    
    def move_up_left(self, piece, board, player_piece,utility=0, enemy_positions=[]):# 1 -> y  0 -> x
        next_position = (piece[0]-1,piece[1]-1)
        if not self.can_move_up(piece, board, player_piece) or not self.can_move_left(piece, board, player_piece) : return False,_,_,0
        if board[piece[0]-1][piece[1]-1] == player_piece: return False,_,_,0
        if board[next_position[0]][next_position[1]] == 0 or board[next_position[0]][next_position[1]] == 3:
            if utility!=0: #Found empty space and can at least take one enemy
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_up_left(next_position, board, player_piece, utility+player_piece,enemy_positions)#enemy found

    def move_up_right(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0]+1,piece[1]-1)
        if not self.can_move_up(piece, board, player_piece) or not self.can_move_right(piece, board, player_piece) : return False,_,_,0
        if board[piece[0]+1][piece[1]-1] == player_piece: return False,_,_,0
        if board[next_position[0]][next_position[1]] == 0 or board[next_position[0]][next_position[1]] == 3: 
            if utility!=0: 
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_up_right(next_position, board, player_piece, utility+player_piece,enemy_positions)#enemy found

    def move_down_left(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0]-1,piece[1]+1)
        if not self.can_move_down(piece, board, player_piece) or not self.can_move_left(piece, board, player_piece) : return False,_,_,0
        if board[piece[0]-1][piece[1]+1] == player_piece: return False,_,_,0
        if board[next_position[0]][next_position[1]] == 0 or board[next_position[0]][next_position[1]] == 3:
            if utility!=0: 
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_down_left(next_position, board, player_piece, utility+player_piece,enemy_positions)#enemy found

    def move_down_right(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0]+1,piece[1]+1)
        if not self.can_move_down(piece, board, player_piece) or not self.can_move_right(piece, board, player_piece) : return False,_,_,0
        if board[piece[0]+1][piece[1]+1] == player_piece: return False,_,_,0
        if board[next_position[0]][next_position[1]] == 0 or board[next_position[0]][next_position[1]] == 3:
            if utility!=0: 
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_down_right(next_position, board, player_piece, utility+player_piece,enemy_positions)#enemy found
    
    def move_up(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0],piece[1]-1)
        up_state = self.can_move_up(piece, board, player_piece)
        if not up_state: return False,_,_,0
        if up_state == "Empty":
            if utility!=0:
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_up(next_position, board, player_piece, utility+player_piece,enemy_positions)

    def move_down(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0],piece[1]+1)
        down_state = self.can_move_down(piece, board, player_piece)
        if not down_state: return False,_,_,0
        if down_state == "Empty":
            if utility!=0:
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_down(next_position, board, player_piece, utility+player_piece,enemy_positions)

    def move_right(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0]+1,piece[1])
        right_state = self.can_move_right(piece, board, player_piece)
        if not right_state: return False,_,_,0
        if right_state == "Empty":
            if utility!=0:
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_right(next_position, board, player_piece, utility+player_piece,enemy_positions)

    def move_left(self, piece, board, player_piece,utility=0, enemy_positions=[]):
        next_position = (piece[0]-1,piece[1])
        left_state = self.can_move_left(piece, board, player_piece)
        if not left_state: return False,_,_,0
        if left_state == "Empty":
            if utility!=0:
                return True,(next_position),enemy_positions,utility
            return False,_,_,0
        enemy_positions.append(next_position)
        return self.move_left(next_position, board, player_piece, utility+player_piece,enemy_positions)
    
    
    def moves(self, player, is_human=False):
        moves_checker = []
        board = copy.deepcopy(self.board)
        player_pieces = self.list_actions(player)
        player_piece = 1
        new_states = []
        if player == "min": player_piece = -1
        for piece in player_pieces:
            
            can_move, move_position, enemies_taken_list, utility = self.move_up(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_down(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_right(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_left(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))

            can_move, move_position, enemies_taken_list, utility = self.move_up_right(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_up_left(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_down_right(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))
            can_move, move_position, enemies_taken_list, utility = self.move_down_left(piece, copy.deepcopy(self.board), player_piece)
            if can_move: new_states.append(State(piece, move_position, utility, enemies_taken_list, copy.deepcopy(board)))

        if is_human:
            return print_possible_moves(new_states)

        new_states = simplify_repited_destins(new_states, self.board)

        for new_state in new_states:
            new_state.print_board_changes()
        
        return new_states

In [ ]:
def get_best_state_index(states, player_piece):
    utility_list = [state.heuristic for state in states]
    max_utility_index = utility_list.index(min(utility_list))
    if player_piece == 1:
        max_utility_index = utility_list.index(max(utility_list))
    return max_utility_index

In [ ]:
def print_possible_moves(states):
    possible_moves = State(copy.deepcopy(states[0].board))
    for state in states:
        possible_moves.board[state.destin[0]][state.destin[1]] = 3
    return possible_moves